In [2]:
# Install `transformers` from master
# ! 👇 apparently this isn't really python code. It only works on jupyter kernal
# %pip install transformers
# %pip list | grep -E 'transformers|tokenizers'
# transformers version at notebook update --- 4.32.1 #//2.11.0
# tokenizers version at notebook update ---  0.13.3 #//0.8.0rc1

import csv
from pathlib import Path
from pprint import pprint

import pandas as pd
import torch
from transformers import (AutoModel, AutoModelForMaskedLM, AutoTokenizer,
                          pipeline)
from utils.LexicalCategories import ADJ_BY_SCALE as ADJ_SETS
from utils.LexicalCategories import ADV_OF_INTEREST as ADV_SETS

pd.set_option('display.max_colwidth', 70)
pd.set_option("display.precision", 5)
MASK_POS = 'adv'
MODEL = 'distilbert-base-uncased'
UNMASK = pipeline('fill-mask', model=MODEL, tokenizer=MODEL, framework='pt')
HITS_PATH = Path(
    '/share/compling/data/sanpi/4_post-processed/RBXadj/bigrams_frq-thr0-01p.25f.pkl.gz')
FRQ_FILTER = Path(
    '/share/compling/projects/sanpi/demo/data/4_post-processed/RBxpos/hit-index_thr0-001p.1f.txt')

RuntimeError: shape '[768]' is invalid for input of size 96

 ## Load sample hit table as data

In [2]:
def _load_hits(dpath=HITS_PATH):

    if '.csv' in dpath.suffixes:
        data = pd.read_csv(dpath)
        data = data.set_index('hit_id').convert_dtypes()

    elif dpath.endswith('pkl.gz'):
        data = pd.read_pickle(dpath)

    print('data loaded')
    pprint(data.columns)
    

    return data

all_data = _load_hits()
all_data.sample(3)

data loaded
Index(['colloc', 'sent_text', 'adv_form', 'adj_form', 'hit_text',
       'text_window', 'sent_id', 'match_id', 'colloc_id', 'token_str',
       'lemma_str', 'context_prev_id', 'context_prev_sent', 'context_next_id',
       'context_next_sent', 'adv_lemma', 'adj_lemma', 'adv_index', 'adj_index',
       'dep_mod', 'json_source', 'utt_len', 'category'],
      dtype='object')


,colloc,sent_text,adv_form,adj_form,hit_text,text_window,sent_id,match_id,colloc_id,token_str,...,context_next_id,context_next_sent,adv_lemma,adj_lemma,adv_index,adj_index,dep_mod,json_source,utt_len,category
hit_id,,,,,,,,,,,,,,,,,,,,,
pcc_eng_29_109.4743_x1747869_06:11-12,particularly_hard,"In a blog post Wednesday, she wrote about a particularly hard nigh...",particularly,hard,particularly hard,wrote about a particularly hard night she experienced,pcc_eng_29_109.4743_x1747869_06,11-12,pcc_eng_29_109.4743_x1747869_06:11-12,"In a blog post Wednesday , she wrote about a particularly hard nig...",...,pcc_eng_29_109.4743_x1747869_07,<NA>,particularly,hard,10,11,"{'node': 'mod', 'contiguous': True, 'relation': 'advmod', 'head': ...",/share/compling/projects/sanpi/demo/data/1_json_grew-matches/RBXad...,24,RBXadj
pcc_eng_29_109.0555_x1741207_081:7-8,very_tight,Puhakka also had Gunderson in a very tight looking anaconda choke ...,very,tight,very tight,Gunderson in a very tight looking anaconda choke,pcc_eng_29_109.0555_x1741207_081,7-8,pcc_eng_29_109.0555_x1741207_081:7-8,Puhakka also had Gunderson in a very tight looking anaconda choke ...,...,pcc_eng_29_109.0555_x1741207_082,<NA>,very,tight,6,7,"{'node': 'mod', 'contiguous': True, 'relation': 'advmod', 'head': ...",/share/compling/projects/sanpi/demo/data/1_json_grew-matches/RBXad...,22,RBXadj
pcc_eng_29_108.6294_x1739422_07:6-7,more_glaring,Here is one of the more glaring:,more,glaring,more glaring,one of the more glaring :,pcc_eng_29_108.6294_x1739422_07,6-7,pcc_eng_29_108.6294_x1739422_07:6-7,Here is one of the more glaring :,...,pcc_eng_29_108.6294_x1739422_08,<NA>,more,glaring,5,6,"{'node': 'mod', 'contiguous': True, 'relation': 'advmod', 'head': ...",/share/compling/projects/sanpi/demo/data/1_json_grew-matches/RBXad...,8,RBXadj


 Apply filters to loaded data, based on frequency as well as manual criteria

In [3]:
def _filter_hits(hits_data):
    # trim data
    filter_hi = pd.read_csv(FRQ_FILTER).squeeze()
    df = hits_data.loc[filter_hi,
                       ['adv_form', 'adj_form', 'adv_lemma', 'adj_lemma',
                        'adv_index', 'adj_index', 'token_str', 'text_window']]
    df = df.assign(
        prev_prev_word=df.index.to_series().apply(
            lambda ix: df.token_str[ix].split()[max(-1,df.adv_index[ix] - 2)]),
        prev_word=df.index.to_series().apply(
            lambda ix: df.token_str[ix].split()[max(-1,df.adv_index[ix] - 1)]),
        next_word=df.index.to_series().apply(
            lambda ix: df.token_str[ix].split()[min(len(df.token_str[ix].split())-1, df.adj_index[ix] + 1)]),
        next_next_word=df.index.to_series().apply(
            lambda ix: df.token_str[ix].split()[min(len(df.token_str[ix].split())-1, df.adj_index[ix] + 2)])
        )
    df.loc[df.prev_word == df.prev_prev_word, 'prev_prev_word'] = ''
    df.loc[df.prev_word == df.adv_form, 'prev_word'] = ''
    df.loc[df.next_word == df.next_next_word, 'next_next_word'] = ''
    df.loc[df.next_word == df.adj_form, 'next_word'] = ''
    manual_criteria = (
        # limit to `token_str` values with no more than 14 spaces ≈ max 15 words
        (df.token_str.str.count(' ') <= 14)
        # NOTE: 'many', 'much' & 'enough' have abnormal syntactic patterns
        & (~df.adv_lemma.str.lower().isin(('enough', 'how', 'much', 'most')))
        & (~df.adj_lemma.str.lower().isin(('many', 'more', 'much', 'enough')))
        # have issues filling position following "much" and "more"--they don't wind up being adverbs but comparative adjectives
        & (~df.prev_word.str.lower().isin(['more', 'much', 'as']))
        & (~df.next_word.str.lower().isin(['as']))
    )
    df = df.loc[manual_criteria, :]

    return df

data = _filter_hits(all_data)
data.sample(1).T

hit_id,pcc_eng_29_108.2074_x1732562_074:11-12
adv_form,extremely
adj_form,pleased
adv_lemma,extremely
adj_lemma,pleased
adv_index,10
adj_index,11
token_str,"Hi Alan , it looks really lovely and I 'm extremely pleased ."
text_window,and I 'm extremely pleased .
prev_prev_word,I
prev_word,'m


In [4]:
print('Adjective lemmas remaining, in order of frequency:')
print(data.value_counts('adj_lemma').to_frame('tokens remaining').to_markdown(floatfmt=',.0f'))

Adjective lemmas remaining, in order of frequency:
| adj_lemma        |   tokens remaining |
|:-----------------|-------------------:|
| good             |                260 |
| important        |                118 |
| easy             |                109 |
| available        |                105 |
| different        |                103 |
| bad              |                 70 |
| sure             |                 64 |
| happy            |                 62 |
| difficult        |                 57 |
| true             |                 47 |
| simple           |                 45 |
| big              |                 45 |
| cool             |                 45 |
| high             |                 44 |
| great            |                 41 |
| excited          |                 40 |
| nice             |                 38 |
| hard             |                 38 |
| interesting      |                 38 |
| popular          |                 37 |
| clear            |     

In [5]:
print('Adverb lemmas remaining, in order of frequency:')
# print(data.value_counts('adv_lemma').to_frame('tokens remaining').to_markdown(floatfmt=',.0f'))
data.value_counts('adv_lemma').to_frame('tokens remaining')

Adverb lemmas remaining, in order of frequency:
| adv_lemma          |   tokens remaining |
|:-------------------|-------------------:|
| very               |                936 |
| not                |                584 |
| more               |                532 |
| so                 |                423 |
| really             |                242 |
| too                |                231 |
| pretty             |                153 |
| quite              |                108 |
| also               |                101 |
| extremely          |                 95 |
| always             |                 78 |
| just               |                 67 |
| less               |                 66 |
| even               |                 65 |
| as                 |                 63 |
| completely         |                 60 |
| still              |                 57 |
| all                |                 54 |
| highly             |                 54 |
| super              |      

In [6]:
data.sample(5).reset_index(drop=True).loc[:, data.columns.str.endswith(('word', 'form', 'str', 'window'))].set_index('text_window')

,adv_form,adj_form,token_str,prev_prev_word,prev_word,next_word,next_next_word
text_window,,,,,,,
Hardly ideal circumstances .,Hardly,ideal,Hardly ideal circumstances .,,.,circumstances,.
therapy session is totally normal .,totally,normal,Feeling nervous about your first therapy session is totally normal .,session,is,.,
""" It is not traditional for the students",not,traditional,""" It is not traditional for the students to leave the area . """,It,is,for,the
proverbial demons is less clear-cut .,less,clear-cut,But how we fight proverbial demons is less clear-cut .,demons,is,.,
called the allegations politically motivated .,politically,motivated,Kobach called the allegations politically motivated .,the,allegations,.,


 ## Mask tokens
 In this version, the data is not pre-masked.
 It consists of sentence strings and target information, including the index of token nodes in the `token_str` values.
 Should be able to use the `adv_index` and/or `adj_index` to create the corresponding masked version of `token_str`

 _Note: for some reason, the initial filter series, `filter_hi`, gets reshaped as the index of `data`, so it cannot be used in place of `data.index.to_series()`_

In [7]:
def pre_mask(df, mask_pos='adv'):
    # df.info()
    mask_ix = df.loc[:, f'{mask_pos}_index']
    # print(mask_ix)
    # #! end index is not included in the returned value! [:mask_ix[h]-1] cuts out the word preceding the adv as well

    return df.index.to_series().apply(
        lambda h: ' '.join(df.token_str[h].split()[:mask_ix[h]]))


def post_mask(df, mask_pos='adv'):
    mask_ix = df.loc[:, f'{mask_pos}_index']

    return mask_ix.index.to_series().apply(
        lambda h: ' '.join(df.token_str[h].split()[mask_ix[h]+1:]))


def mask(df: pd.DataFrame, pos='adv'):
    # sourcery skip: use-fstring-for-concatenation
    pre_col = pre_mask(df, pos)
    post_col = post_mask(df, pos)
    return df.assign(masked=pre_col + ' [MASK] ' + post_col)


data = mask(data, MASK_POS)

In [8]:
data.sample(9).rename(columns={'adv_form':'original token'}).set_index('original token')[['masked']]

,masked
original token,
more,It could have been deeply improved with [MASK] suspenseful sex
so,I was [MASK] glad it was n't me .
pretty,But Boyd also flung a few [MASK] deep passes .
Really,[MASK] childish name-calling .
more,Is there anything [MASK] intimidating than moving to a new place ?
very,It was a [MASK] brief speech -- and not a pleasant one .
more,They want to make lives of the poor even [MASK] miserable .
absolutely,It 's [MASK] FREE !
So,[MASK] cute !


 ## *Unmask* Tokens
 Data is loaded and now _masked_ sentences are a column in the dataframe.
 Can use `UNMASK` pipeline to get probabilities of tokens at masked position.

 ## For just 1 input

 Select sample sentence and run pipeline

In [9]:
selected_data = data.loc[:, ['adv_form', 'adj_form', 'text_window', 'masked', 'prev_word', 'next_word']]
cat = 'absurd_intense'
try:
    sent_row = selected_data.loc[data.adv_lemma.isin(ADV_SETS[cat]), :].sample(1)
except ValueError: 
    sent_row = selected_data.sample(1)

sent = sent_row.masked.squeeze()
sent_row.transpose()

hit_id,pcc_eng_29_108.4740_x1736939_04:6-7
adv_form,oddly
adj_form,specific
text_window,"four is an oddly specific number , no"
masked,"Seventy - four is an [MASK] specific number , no ?"
prev_word,an
next_word,number


 get top 10 predictions for masked position

In [10]:
K1=50
results = UNMASK(sent, top_k=K1)
rdf = pd.DataFrame(results).sort_values('score', ascending=False)
rdf.set_index('token_str')

,score,token,sequence
token_str,,,
extremely,0.09949,5186,"seventy - four is an extremely specific number, no?"
unusually,0.09207,12890,"seventy - four is an unusually specific number, no?"
exceptionally,0.04772,17077,"seventy - four is an exceptionally specific number, no?"
infinitely,0.02706,25773,"seventy - four is an infinitely specific number, no?"
alphabet,0.02445,12440,"seventy - four is an alphabet specific number, no?"
arbitrary,0.01863,15275,"seventy - four is an arbitrary specific number, no?"
apparently,0.01784,4593,"seventy - four is an apparently specific number, no?"
actual,0.01173,5025,"seventy - four is an actual specific number, no?"
overly,0.01111,15241,"seventy - four is an overly specific number, no?"


 load lexical categories and compare

In [11]:
def add_lexcats(results: pd.DataFrame,
                lex_dict: dict,
                original_forms: pd.Series) -> pd.DataFrame:
    if results.index.name == 'token_str':
        results = results.reset_index()
    cat_start_index = len(results.columns)
    for lex_type, lex_set in lex_dict.items():
        results.loc[:, lex_type] = results.token_str.isin(lex_set)
    results = results.assign(any_defined_cat=rdf.iloc[:, cat_start_index:].apply(any, axis=1),
                             # match_masked_POS=results.token_str.isin(original_forms)
                             )
    return results.replace({True: 1, False: 0})


if MASK_POS == 'adj':
    lexcat_dict = ADJ_SETS
elif MASK_POS == 'adv':
    # for adv_type, adv_set in ADV_SETS.items():
    #     rdf.loc[:, adv_type] = rdf.token_str.isin(adv_set)
    # rdf = rdf.assign(match_masked_POS=rdf.token_str.isin(data.adv_form))
    lexcat_dict = ADV_SETS

rdf = add_lexcats(rdf, lexcat_dict, data[f'{MASK_POS}_form'])
if rdf.index.name != 'token_str':
    rdf = rdf.set_index('token_str')
print(f'## input:\n   > {sent}')

## input:
   > Seventy - four is an [MASK] specific number , no ?


In [12]:
rdf.loc[:, ['score']].join(rdf.iloc[:, 3:]).head(4)


,score,sufficient,min_threshold,max_threshold,moderate,precise_intense,compare,weak_intense,meas_intense,mod_intense,...,excess_intense,max_intense,EDM_Morzycki,Mmod_SoltWilson,negative,positive,NPS,PS_rescuer,PPS,any_defined_cat
token_str,,,,,,,,,,,,,,,,,,,,,
extremely,0.09949,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
unusually,0.09207,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
exceptionally,0.04772,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
infinitely,0.02706,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [13]:
lex_counts = rdf.iloc[:,3:].sum().sort_values(ascending=False).astype('uint8').to_frame('count')
lex_counts = lex_counts.assign(found=pd.Series({col: rdf.loc[rdf[col].apply(bool), col].index.to_list() for col in rdf.columns[3:]}))
lex_counts = lex_counts.assign(composite_score=lex_counts.found.apply(lambda words: rdf.loc[words, 'score'].sum()).round(3), 
                  found=lex_counts.found.apply(lambda x:', '.join(x)))
lex_counts.sort_values('composite_score', ascending=False)


,count,found,composite_score
any_defined_cat,14,"extremely, unusually, exceptionally, infinitely, apparently, overl...",0.338
expectation_intense,3,"unusually, exceptionally, incredibly",0.148
mod_intense,3,"extremely, entirely, especially",0.111
time,1,infinitely,0.027
reporting,1,apparently,0.018
excess_intense,1,overly,0.011
absurd_intense,1,oddly,0.009
observable,2,"obviously, intentionally",0.008
max_intense,1,entirely,0.006
good_intense,1,amazingly,0.003


 ## Across multiple inputs

 Apply across all masked sentences and collect results

In [14]:
pprint([{'hit_id': i,
         f'original_{MASK_POS}': selected_data.loc[i, f'{MASK_POS}_form'],
         'text_window': selected_data.text_window[i],
         'masked_str': selected_data.masked[i],
         'scores': UNMASK(selected_data.masked[i], top_k=3)}
        for i in selected_data.sample(3).index])


[{'hit_id': 'pcc_eng_29_108.0099_x1729367_17:1-2',
  'masked_str': ' [MASK] afraid of contact .',
  'original_adv': 'Not',
  'scores': [{'score': 0.14227072894573212,
              'sequence': 'not afraid of contact.',
              'token': 2025,
              'token_str': 'not'},
             {'score': 0.09962216764688492,
              'sequence': 'too afraid of contact.',
              'token': 2205,
              'token_str': 'too'},
             {'score': 0.038818925619125366,
              'sequence': 'never afraid of contact.',
              'token': 2196,
              'token_str': 'never'}],
  'text_window': 'Not afraid of contact .'},
 {'hit_id': 'pcc_eng_29_109.0330_x1740841_18:3-4',
  'masked_str': 'Is this [MASK] true ?',
  'original_adv': 'all',
  'scores': [{'score': 0.20503869652748108,
              'sequence': 'is this all true?',
              'token': 2035,
              'token_str': 'all'},
             {'score': 0.1602722853422165,
              'sequence': 'is t

In [15]:
selected_data = selected_data.sample(300) # HACK: #! temporary. REMOVE


def get_scores(masked, top_k=10):
    return {word_fill['token_str']: word_fill['score'] for word_fill in UNMASK(masked, top_k=top_k)}


def sum_lexcats(scores: dict, lex_dict: dict):
    results = pd.Series(scores).to_frame('score')
    # results.index.name = 'filler_str'
    # results = results.reset_index()
    lexcat_sums = pd.Series(dtype='float')
    for lex_type, lex_set in lex_dict.items():
        # return results.loc[results.index.isin(lex_set), 'score'].sum()
        lexcat_sums.loc[f'{lex_type}_total'] = results.loc[results.index.isin(
            lex_set), 'score'].sum().round(5)
    return lexcat_sums

In [16]:
score_df = selected_data.assign(scores=selected_data.masked.apply(get_scores, top_k=50))

In [17]:
cat_score_df = score_df.join(score_df.scores.apply(sum_lexcats, lex_dict=ADV_SETS)).convert_dtypes()
is_float = cat_score_df.dtypes=='Float64'
cat_score_df.loc[:, is_float] = cat_score_df.loc[:, is_float].apply(pd.to_numeric, downcast='float')
# cat_score_df
cat_score_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 300 entries, pcc_eng_29_108.4104_x1735877_12:09-10 to pcc_eng_29_109.2648_x1744516_34:5-6
Data columns (total 42 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   adv_form                   300 non-null    string 
 1   adj_form                   300 non-null    string 
 2   text_window                300 non-null    string 
 3   masked                     300 non-null    string 
 4   prev_word                  300 non-null    string 
 5   next_word                  300 non-null    string 
 6   scores                     300 non-null    object 
 7   sufficient_total           300 non-null    Float32
 8   min_threshold_total        300 non-null    Float32
 9   max_threshold_total        300 non-null    Float32
 10  moderate_total             300 non-null    Float32
 11  precise_intense_total      300 non-null    Float32
 12  compare_total              300 non-null    

In [18]:
stats = cat_score_df.describe().round(3).T.sort_values('mean', ascending=False)
stats

,count,mean,std,min,25%,50%,75%,max
weak_intense_total,300.0,0.106,0.133,0.0,0.009,0.050,0.166,0.657
compare_total,300.0,0.072,0.201,0.0,0.000,0.003,0.022,0.992
sufficient_total,300.0,0.070,0.174,0.0,0.000,0.004,0.043,0.995
PS_rescuer_total,300.0,0.051,0.155,0.0,0.000,0.004,0.021,0.992
PPS_total,300.0,0.051,0.094,0.0,0.001,0.013,0.055,0.647
Mmod_SoltWilson_total,300.0,0.049,0.094,0.0,0.000,0.011,0.045,0.645
negative_total,300.0,0.036,0.118,0.0,0.000,0.003,0.021,0.987
mod_intense_total,300.0,0.034,0.062,0.0,0.001,0.011,0.041,0.549
time_total,300.0,0.028,0.074,0.0,0.000,0.006,0.027,0.856
max_intense_total,300.0,0.022,0.045,0.0,0.000,0.003,0.020,0.310


In [19]:
adv_sets = pd.Series(dtype='string')
for adv_cat in stats.index:
    adv_cat = adv_cat.replace('_total', '')
    adv_sets[adv_cat] = ', '.join(ADV_SETS[adv_cat])
adv_df = adv_sets.to_frame(f'{MASK_POS} list')
adv_df.index.name = 'category'
pd.set_option('display.max_colwidth', 120)

In [69]:
adv_category = pd.Series({k: ', '.join(v) for k,v in ADV_SETS.items()}).sort_index()
adv_cat_df = adv_category.to_frame('words')
adv_cat_df.index.name='adv_category'


In [70]:
# adv_cat_df = adv_cat_df.assign(cat_type = adv_category.replace(adv_category.str.endswith(('intense', 'Morzycki')), 'intensifier'))
# adv_cat_df.cat_type.where(cond=adv_cat_df.cat_type == 'non-intensifier', other='intensifier', inplace=True)
# adv_cat_df
intense = adv_category.index.to_series().str.endswith(('intense','Morzycki','x_threshold'))
adv_cat_df.loc[:, '+intense'] = intense
adv_cat_df


,words,+intense
adv_category,,
EDM_Morzycki,"simply, flat-out, outright, absolutely, downright, full-on, just, positively, balls-out, straight-up, out-and-out",True
Mmod_SoltWilson,"fairly, kinda, somewhat, rather, sorta, pretty",False
NPS,"ever, exactly, yet",False
PPS,"fairly, kinda, somewhat, utterly, rather, sorta, pretty",False
PS_rescuer,"only, enough, even, too",False
absurd_intense,"strangely, bizarrely, ridiculously, peculiarly, weirdly, freakily, absurdly, freakishly, curiously, oddly",True
bad_intense,"unfortunately, dismally, disappointingly, badly",True
compare,"more, most, less, comparatively, least, as",False
consume_intense,"insatiably, greedily, voraciously",True


In [71]:
adv_cat_df = adv_cat_df.assign(
    req_scale=adv_cat_df.index.str.endswith(
        ('compare', 'moderate', 'sufficient', 'Mmod_SoltWilson', 'threshold', 'intense'))
)
adv_cat_df


,words,+intense,req_scale
adv_category,,,
EDM_Morzycki,"simply, flat-out, outright, absolutely, downright, full-on, just, positively, balls-out, straight-up, out-and-out",True,False
Mmod_SoltWilson,"fairly, kinda, somewhat, rather, sorta, pretty",False,True
NPS,"ever, exactly, yet",False,False
PPS,"fairly, kinda, somewhat, utterly, rather, sorta, pretty",False,False
PS_rescuer,"only, enough, even, too",False,False
absurd_intense,"strangely, bizarrely, ridiculously, peculiarly, weirdly, freakily, absurdly, freakishly, curiously, oddly",True,True
bad_intense,"unfortunately, dismally, disappointingly, badly",True,True
compare,"more, most, less, comparatively, least, as",False,True
consume_intense,"insatiably, greedily, voraciously",True,True


In [76]:
adv_cat_table = adv_cat_df[['+intense', 'req_scale', 'words']].to_markdown()

Path('/home/arh234/projects/sanpi/info/adv-cat_table-23-09-08.md').write_text(f'# Adverb Categories\n\n{adv_cat_table}')

9492

In [73]:
cat_score_df.sample(2)

,adv_form,adj_form,text_window,masked,prev_word,next_word,scores,sufficient_total,min_threshold_total,max_threshold_total,...,unease_intense_total,excess_intense_total,max_intense_total,EDM_Morzycki_total,Mmod_SoltWilson_total,negative_total,positive_total,NPS_total,PS_rescuer_total,PPS_total
hit_id,,,,,,,,,,,,,,,,,,,,,
pcc_eng_29_109.5996_x1749878_65:5-6,pretty,young,", that 's pretty young .","Yeah , that 's [MASK] young .",'s,.,"{'pretty': 0.5080512166023254, 'too': 0.10320878773927689, 'really': 0.04407912865281105, 'very': 0.0402987524867057...",0.14177,0.0,0.0,...,0,0.0,0.00201,0.00119,0.54555,0.0028,0.00144,0.0,0.10321,0.54555
pcc_eng_29_109.2648_x1744516_34:5-6,remotely,realistic,"It 's not remotely realistic , "" he",""" It 's not [MASK] realistic , "" he said .",not,",","{'very': 0.15076076984405518, 'exactly': 0.12407334893941879, 'too': 0.10400662571191788, 'quite': 0.078553706407547...",0.13753,0.0,0.03587,...,0,0.02998,0.12642,0.0092,0.00601,0.0,0.01852,0.13192,0.12115,0.00601


In [74]:
scores_by_adv = pd.DataFrame.from_records((cat_score_df.scores), index=cat_score_df.index)
scores_by_adv.loc['total', :] = scores_by_adv.sum()
scores_by_adv.T.sort_values('total', ascending=False)

hit_id,pcc_eng_29_108.4104_x1735877_12:09-10,pcc_eng_29_108.0914_x1730699_44:3-4,pcc_eng_29_109.0678_x1741426_17:12-13,pcc_eng_29_108.2083_x1732574_17:8-9,pcc_eng_29_108.3720_x1735266_51:3-4,pcc_eng_29_109.5996_x1749878_59:6-7,pcc_eng_29_108.4794_x1737020_15:10-11,pcc_eng_29_109.3668_x1746155_83:13-14,pcc_eng_29_109.5831_x1749611_15:11-12,pcc_eng_29_108.2695_x1733607_17:4-5,...,pcc_eng_29_109.4947_x1748195_21:3-4,pcc_eng_29_109.3836_x1746418_082:3-4,pcc_eng_29_108.1860_x1732220_26:4-5,pcc_eng_29_108.4585_x1736672_46:6-7,pcc_eng_29_109.3412_x1745737_41:6-7,pcc_eng_29_108.2616_x1733488_16:7-8,pcc_eng_29_108.2413_x1733128_03:1-2,pcc_eng_29_108.5679_x1738439_176:8-9,pcc_eng_29_109.2648_x1744516_34:5-6,total
very,0.10623,0.20558,0.00725,0.08908,NaN,0.33679,0.00579,0.36425,0.01833,0.19588,...,0.05275,0.00460,0.01478,0.00188,0.02826,0.00152,0.12387,0.01665,0.15076,1.84971e+01
more,NaN,0.00304,NaN,0.00562,NaN,NaN,0.00610,0.01741,0.00511,0.01899,...,NaN,NaN,NaN,NaN,0.37632,0.00164,0.00825,0.03024,0.00382,1.29377e+01
too,0.00166,0.01035,0.00311,0.03895,NaN,0.15853,NaN,0.01174,0.00363,0.04746,...,0.05965,0.14533,0.09557,0.00194,0.04206,NaN,NaN,NaN,0.10401,1.27260e+01
not,0.00275,NaN,0.01318,0.01919,NaN,0.00150,0.52274,0.01153,NaN,NaN,...,NaN,0.01685,0.16666,0.00504,NaN,NaN,0.01058,NaN,NaN,9.41714e+00
pretty,0.00208,0.00324,NaN,NaN,NaN,0.15799,NaN,0.00759,0.27420,0.03983,...,0.20273,0.03095,0.05195,NaN,NaN,NaN,0.03435,0.01334,0.00601,8.32201e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
adult,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.31247e-06
explicit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.29792e-06
using,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.27568e-06
unlawful,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.10245e-06


In [75]:
# scores_by_adv=scores_by_adv.join(data.loc[:, ['adj_form', 'adj_lemma', 'adv_form', 'adv_lemma']])